In [1]:
import pandas as pd

In [2]:
df_historical_data = pd.read_csv('fifa_world_cup_historical_data.csv')
df_fixture = pd.read_csv('fifa_2022_world_cup_fixtures.csv')
df_missing_data = pd.read_csv('fifa_worldcup_missing_data.csv')

In [3]:
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
500,Russia,2–2 (a.e.t.),Croatia,2018
501,France,1–0,Belgium,2018
502,Croatia,2–1 (a.e.t.),England,2018
503,Belgium,2–0,England,2018


In [4]:
# Remove spaces at the beginning and at the end of the string

df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

In [5]:
df_missing_data.dropna(inplace=True)
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
467,Brazil,2–0,Costa Rica,2018
468,Serbia,1–2,Switzerland,2018
469,Serbia,0–2,Brazil,2018
471,Germany,0–1,Mexico,2018


In [6]:
# deleting match with walk over
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                  df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

# cleanning score and home/away columns
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
df_historical_data['home'] = df_historical_data['home'].str.strip() # clean blank spaces: Yugoslavia twice
df_historical_data['away'] = df_historical_data['away'].str.strip()

# splitting score columns into home and away goals and dropping score column
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
467,Brazil,Costa Rica,2018,2,0,2
468,Serbia,Switzerland,2018,1,2,3
469,Serbia,Brazil,2018,0,2,2
471,Germany,Mexico,2018,0,1,1


In [7]:
df_historical_data.to_csv('clean_fifa_worldcup_matches.csv',index=False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv',index=False)

In [8]:
# verify number of matches per competition
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 16
2006 16
2010 64
2014 64
2018 64


In [9]:
# verify data collected for a team
print(df_historical_data[df_historical_data['HomeTeam'].str.contains('Turkey')])
print(df_historical_data[df_historical_data['AwayTeam'].str.contains('Turkey')])

   HomeTeam     AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
83   Turkey  South Korea  1954          7          0           7
         HomeTeam AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
80   West Germany   Turkey  1954          4          1           5
84   West Germany   Turkey  1954          7          2           9
292       Senegal   Turkey  2002          0          1           1
295   South Korea   Turkey  2002          2          3           5
294        Brazil   Turkey  2002          1          0           1
287         Japan   Turkey  2002          0          1           1
